In [1]:
# Importing the important libraries:
import pandas as pd
import numpy as np

In [2]:
# Reading the data into pandas:
df = pd.read_csv("glassdoor_jobs.csv")

In [3]:
# Explore the structure of the data:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         956 non-null    int64  
 1   Job Title          956 non-null    object 
 2   Salary Estimate    956 non-null    object 
 3   Job Description    956 non-null    object 
 4   Rating             956 non-null    float64
 5   Company Name       956 non-null    object 
 6   Location           956 non-null    object 
 7   Headquarters       956 non-null    object 
 8   Size               956 non-null    object 
 9   Founded            956 non-null    int64  
 10  Type of ownership  956 non-null    object 
 11  Industry           956 non-null    object 
 12  Sector             956 non-null    object 
 13  Revenue            956 non-null    object 
 14  Competitors        956 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 112.2+ KB


In [4]:
# view the immediate five rows:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


## Data cleansing

In [5]:
       #salary parsing
        #company name text only
        #state field
        #age of company
        #parsing of job description (python, etc.)

#### salary parsing

In [6]:
# Removing the -1 salary:
df = df[df['Salary Estimate'] != '-1']

In [7]:
df['Salary Estimate'].head()

0     $53K-$91K (Glassdoor est.)
1    $63K-$112K (Glassdoor est.)
2     $80K-$90K (Glassdoor est.)
3     $56K-$97K (Glassdoor est.)
4    $86K-$143K (Glassdoor est.)
Name: Salary Estimate, dtype: object

In [8]:
# spliting the salary column:
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

In [9]:
salary.head()

0     $53K-$91K 
1    $63K-$112K 
2     $80K-$90K 
3     $56K-$97K 
4    $86K-$143K 
Name: Salary Estimate, dtype: object

In [10]:
# Removing k and the $ sign:
minus_kd = salary.apply(lambda x: x.replace('K', "").replace('$',''))

In [11]:
minus_kd.head()

0     53-91 
1    63-112 
2     80-90 
3     56-97 
4    86-143 
Name: Salary Estimate, dtype: object

In [12]:
# creating column employer provided salary:
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

In [13]:
df['employer_provided'].head()

0    0
1    0
2    0
3    0
4    0
Name: employer_provided, dtype: int64

In [14]:
# creating column for hours:
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

In [15]:
df['hourly'].head()

0    0
1    0
2    0
3    0
4    0
Name: hourly, dtype: int64

In [16]:
# separating hr and employer provided salary:
min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

In [17]:
min_hr.head()

0     53-91 
1    63-112 
2     80-90 
3     56-97 
4    86-143 
Name: Salary Estimate, dtype: object

In [18]:
# separating the min from the max salaries:
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))

In [19]:
df['min_salary'].head()

0    53
1    63
2    80
3    56
4    86
Name: min_salary, dtype: int64

In [20]:
# to know the min_salary data type:
df['min_salary'].dtype

dtype('int64')

In [21]:
# separating the max from the min salaries:
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))

In [22]:
df['max_salary'].head()

0     91
1    112
2     90
3     97
4    143
Name: max_salary, dtype: int64

In [23]:
# finding the average salary column:
df['avg_salary'] = (df.min_salary + df.max_salary)/2

In [24]:
df['avg_salary'].head()

0     72.0
1     87.5
2     85.0
3     76.5
4    114.5
Name: avg_salary, dtype: float64

#### company name text only

In [25]:
# separating the company name from the rating:
df['company_name'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else  x['Company Name'][:-4], axis = 1)

In [26]:
df['company_name'].head()

0                        Tecolote Research
1    University of Maryland Medical System
2                                  KnowBe4
3                                     PNNL
4                       Affinity Solutions
Name: company_name, dtype: object

#### state field

In [27]:
# separate the state from the job:
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])

In [28]:
# the count of job state:
df.job_state.value_counts() 

 CA             151
 MA             103
 NY              72
 VA              41
 IL              40
 MD              35
 PA              33
 TX              28
 NC              21
 WA              21
 NJ              17
 FL              16
 OH              14
 TN              13
 CO              11
 DC              11
 IN              10
 UT              10
 WI              10
 AZ               9
 MO               9
 AL               8
 DE               6
 MI               6
 KY               6
 GA               6
 IA               5
 CT               5
 LA               4
 NE               4
 OR               4
 NM               3
 KS               3
 ID               2
 MN               2
 Los Angeles      1
 RI               1
 SC               1
Name: job_state, dtype: int64

In [29]:
# to check if the same state is with headquarter:
df['same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)

In [30]:
df['same_state'].head()

0    0
1    0
2    1
3    1
4    1
Name: same_state, dtype: int64

#### age of company

In [31]:
# extract the age from the founded column:
df['age'] = df.Founded.apply(lambda x: x if x < 1 else 2020 - x)

In [32]:
df['age'].head()

0    47
1    36
2    10
3    55
4    22
Name: age, dtype: int64

#### parsing of job description (python, etc.)

In [33]:
# checking out for python in the job description:
df['python_yn'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [34]:
df['python_yn'].head()

0    1
1    1
2    1
3    1
4    1
Name: python_yn, dtype: int64

In [35]:
# counts of python in the job description:
df.python_yn.value_counts()

1    392
0    350
Name: python_yn, dtype: int64

In [36]:
# checking out for r studio:
df['R_yn'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

In [37]:
df['R_yn'].head()

0    0
1    0
2    0
3    0
4    0
Name: R_yn, dtype: int64

In [38]:
# count of R programing language in the job description:
df.R_yn.value_counts()

0    740
1      2
Name: R_yn, dtype: int64

In [39]:
#checking out for spark:
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [40]:
#count of spark in the job description:
df.spark.value_counts()

0    575
1    167
Name: spark, dtype: int64

In [41]:
# checking out for aws:
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower()else 0)

In [42]:
#count of aws in the job description:
df.aws.value_counts()

0    566
1    176
Name: aws, dtype: int64

In [43]:
# checking out for excel:
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

In [44]:
#count of excel in the job description:
df.excel.value_counts()

1    388
0    354
Name: excel, dtype: int64

In [45]:
# dropping the unamed column:
cleaned_df = df.drop(['Unnamed: 0'], axis = 1)

In [46]:
# save this cleaned dataframe to csv file with a new name:
cleaned_df.to_csv('salary_data_cleaned.csv',index = False)